In [1]:
import torch
from torch import nn

In [2]:
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,1))
X=torch.rand(size=(2,4))
net(X)

tensor([[-0.3571],
        [-0.2778]], grad_fn=<AddmmBackward>)

In [3]:
net.state_dict()

OrderedDict([('0.weight', tensor([[-0.3239,  0.2691,  0.3630,  0.1693],
                      [-0.2764, -0.0719, -0.4766, -0.2535],
                      [-0.3111,  0.0625, -0.4074, -0.2234],
                      [ 0.2071,  0.0189, -0.4473,  0.1998],
                      [-0.1658, -0.3851,  0.0630,  0.2025],
                      [-0.4799, -0.0166, -0.2771, -0.1344],
                      [-0.2123,  0.2754,  0.2844, -0.3750],
                      [ 0.0723, -0.2770,  0.3301,  0.0969]])),
             ('0.bias',
              tensor([ 0.4576, -0.3863,  0.0071,  0.3652, -0.2898,  0.2208, -0.4588,  0.4863])),
             ('2.weight',
              tensor([[-0.1603,  0.0969, -0.0062,  0.1115, -0.0597,  0.1033, -0.0052,  0.1106]])),
             ('2.bias', tensor([-0.3102]))])

In [4]:
[(name,param.shape) for name ,param in net[0].named_parameters()]

[('weight', torch.Size([8, 4])), ('bias', torch.Size([8]))]

In [6]:
print(*[(name,param.shape) for name ,param in net[0].named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))


In [7]:
net.state_dict()['2.bias'].data

tensor([-0.3102])

In [8]:
net.state_dict()['2.weight'].data

tensor([[-0.1603,  0.0969, -0.0062,  0.1115, -0.0597,  0.1033, -0.0052,  0.1106]])

In [9]:
def block1():
    return nn.Sequential(nn.Linear(4,8),nn.ReLU(),nn.Linear(8,4),nn.ReLU())

def block2():
    net=nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}',block1())
    return net

In [10]:
rgnet=nn.Sequential(block2(),nn.Linear(4,1))
rgnet(X)

tensor([[0.2980],
        [0.2980]], grad_fn=<AddmmBackward>)

In [11]:
rgnet

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)

In [12]:
rgnet[0]

Sequential(
  (block 0): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
  (block 1): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
  (block 2): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
  (block 3): Sequential(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=4, bias=True)
    (3): ReLU()
  )
)

In [13]:
rgnet[1]

Linear(in_features=4, out_features=1, bias=True)

In [18]:
rgnet[0][1][0].bias.data

tensor([-0.4024,  0.3458,  0.1588, -0.4379, -0.4409, -0.2001, -0.1342, -0.0927])

In [20]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight,mean=0,std=0.01)
        nn.init.zeros_(m.bias)


In [22]:
net


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [25]:
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([-0.0109, -0.0061,  0.0006, -0.0020]), tensor(0.))

In [26]:
def init_constant(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,1)
        nn.init.zeros_(m.bias)

In [29]:
net.apply(init_constant)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [30]:
def xavier(m):
    if type(m)==nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m)==nn.Linear:
        nn.init.constant_(m.weight,42)

In [36]:
net[0].apply(xavier)
net[2].apply(init_42)
net[0].weight.data[0],net[2].weight.data

(tensor([-0.4026, -0.4939, -0.5718, -0.4676]),
 tensor([[42., 42., 42., 42., 42., 42., 42., 42.]]))

In [50]:
def my_init(m):
    if type(m)==nn.Linear:
        print('Init',*[(name,param.shape) for name ,param in m.named_parameters()][0])
        nn.init.uniform_(m.weight,-10,10)
        m.weight.data *= m.weight.data.abs() >=5

In [47]:
net[0].weight.shape

torch.Size([8, 4])

In [49]:
print('Init',[(name,param.shape) for name ,param in net.named_parameters()])

Init [('0.weight', torch.Size([8, 4])), ('0.bias', torch.Size([8])), ('2.weight', torch.Size([1, 8])), ('2.bias', torch.Size([1]))]


In [42]:
print('Init',*[(name,param.shape) for name ,param in net.named_parameters()])

Init ('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [48]:
print('Init',*[(name,param.shape) for name ,param in net.named_parameters()][0])

Init 0.weight torch.Size([8, 4])


In [52]:
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000,  0.0000, -6.3042,  9.6194],
        [-9.8569, -9.4396, -5.6798, -0.0000]], grad_fn=<SliceBackward>)

In [54]:
net[0].weight.data[:]+=1
net[0].weight.data[0,0]=42
net[0].weight.data[0]

tensor([42.0000,  2.0000, -4.3042, 11.6194])

In [55]:
shared=nn.Linear(8,8)
net=nn.Sequential(nn.Linear(4,8),nn.ReLU(),shared,nn.ReLU(),shared,nn.ReLU(),nn.Linear(8,1))

In [56]:
net(X)

tensor([[-0.4665],
        [-0.4583]], grad_fn=<AddmmBackward>)

In [57]:
net[2].weight.data[0]==net[4].weight.data[0]

tensor([True, True, True, True, True, True, True, True])

In [58]:
net[2].weight.data[0,0]=100
net[2].weight.data[0]==net[4].weight.data[0]

tensor([True, True, True, True, True, True, True, True])